In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Projectile Motion: The Trajectory of a Cannon Shell

Going forward from the bicyle problem, we turn our attention to two-dimensional projectile motion.  Our specific example will be the shell shot by a large cannon.

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo("hlW6hZkgmkA", playlist="hlW6hZkgmkA", autoplay=1, loop=1)

In the absence of air-resistance, the equations of motion (Newton's 2nd Law) are:

$$
\begin{eqnarray}
\frac{d^2 x}{dt^2} =& 0 \\
\frac{d^2 y}{dt^2} =& -g
\end{eqnarray}
$$

These are second-order differential equations.  A standard technique for solving higher order differential equations is to rewrite them as a system of first order differential equations.  That is,

$$
\begin{eqnarray}
\frac{d x}{dt} =& v_x \\
\frac{d v_x}{dt} =& 0 \\
\frac{d y}{dt} =& v_y \\
\frac{d v_y}{dt} =& -g
\end{eqnarray}
$$

There are now four equations written out for the variables $x, y, v_x, v_y$.  As before, we can solve this problem by using Euler's method:

$$
\begin{align}
x_{i+1} &= x_i + v_{x,i} \Delta t \\
v_{x,i+1} &= v_{x,i} \\
y_{i+1} &= y_i + v_{y,i} \Delta t \\
v_{y,i+1} &= v_{y,i} - g \Delta t
\end{align}
$$

This algorithm allows us to estimate the position and velocity of a projectile given the intial values of $x, y, v_x$, and $v_y$.
 

Like in the bicycle problem, air resistance is important. To introduce air resistance we can model the drag as

$$ F_{drag} = - B_2 v^2 $$

like we did in the bicycle problem where $v$ is the magnitude of the velocity vector. Since force is vector so we need to split this into horizontal and vertical components.

This force is a vector and always directed in the opposite direction of the velocity vector.

\begin{align}
F_{drag,x} = F_{drag} \cos \theta = F_{drag} (v_x / v) \\
F_{drag,y} = F_{drag} \sin \theta = F_{drag} (v_y / v)
\end{align}

we can write

$$
\begin{align}
F_{drag, x} = - B_2 v v_x \\
F_{drag, y} = - B_2 v v_y 
\end{align}
$$

We can add these drag forces to our algorithm like we did last lecture.


\begin{align}
x_{i+1} &= x_i + v_{x,i} \Delta t \\
v_{x,i+1} &= v_{x,i} - \frac{B_2 v v_x}{m} \Delta t \\
y_{i+1} &= y_i + v_{y,i} \Delta t \\
v_{y,i+1} &= v_{y,i} - g \Delta t - \frac{B_2 v v_x}{m} \Delta t
\end{align}

### Simulation of position vs. time for a cannon ball

#### Initialize variables

In [ ]:
# acceleration due to gravity, m/s^2
g = 9.81
# time step, s
dt = 0.1
# max time, s
tmax = 200

In this problem, the variables are `x`, `y`, `vx`, and `vy`.  In the code below we create numpy arrays for `x` and `y` since we want to record the values of position for over many different timesteps.  

The velocity components `vx` and `vy` are simple variables that represent only the value of the velocity at the *current* time step.

Finally, in this solution we don't even have time, `t`, listed explicited.  There is still a time step, `dt`, and a maximum time, `tmax`, but no array for time.

Depending on the problem we are solving, we can choose what variables we want to record. If you preferred, you can have arrays for `vx`, `vy`, and `t` as well.

#### Compute steps

In [ ]:
def update(i, x, y, vx, vy, Bm):
    # update positions
    x[i+1] = x[i] + vx*dt
    y[i+1] = y[i] + vy*dt
        
    # calculate velocity magnitude
    v = np.sqrt(vx**2 + vy**2)
        
    # compute air drag
    Fdrag_x = -(Bm)*v*vx
    Fdrag_y = -(Bm)*v*vy
        
    # update velocities
    vx_new = vx + Fdrag_x * dt
    vy_new = vy + (-g + Fdrag_y) * dt
    
    return vx_new, vy_new

In [ ]:
def solve(theta=45, v0=700, Bm=4e-5):
    """
    Given 
        theta, firing angle (degrees)
        v0, initial velocity (m/s)
        Bm, drag coefficient per unit mass (1/m)
    Compute
        x,y trajectory of a cannon shell
    """
    
    # Allocate memory for arrays and set to zero
    N = round(tmax / dt)
    x = np.zeros(N)
    y = np.zeros(N)
    
    # Initial values
    x[0] = 0
    y[0] = 0
    vx = v0*np.cos(np.deg2rad(theta))
    vy = v0*np.sin(np.deg2rad(theta))

    # Calculate the solution
    for i in range(N-1):
        vx, vy = update(i, x, y, vx, vy, Bm)
        
        # stop when y <= 0
        if y[i+1] <= 0:
            break

    # truncate arrays to when shell has hit the ground (y<=0)
    N = i+2
    x = x[:N]
    y = y[:N]

    # interpolate position if shell is below ground level
    if y[i+1] < 0:
        r = -y[i]/y[i+1]
        x[i+1] = (x[i]+r*x[i+1])/(r+1)
        y[i+1] = 0 

    return x, y

Notice that the main `for` loop has an `if` statement inside. This code checks if the projectile has hit the ground.  The truncation and interpolation steps make it so the trajectory ends exactly at the ground. 

#### Plotting routine

In [ ]:
def plot(x, y, label='', style='-'):
    plt.title('Trajectory of cannon shell')
    # show results in km
    plt.plot(x/1000, y/1000, style, label=label)
    plt.xlabel('x (km)')
    plt.ylabel('y (km)')

Notice we are plotting `x` vs `y`.  For 2D projectile problems this is the best way of visualing the solution. 

If you wanted, you could also plot `x` vs `t` and `y` vs `t` as two separate plots.  In that case, you would need to calculate and store the time `t` as an array.

#### Driver code

In [ ]:
fig, axes = plt.subplots()

x, y = solve(theta=theta, Bm=4e-5)
plot(x, y, label='With air drag')

plt.show()

#### Exercise

> 1. Run the again model for without air drag. How does this change the range?
> 2. Run the model for several different firing angles. For what angle is the range maximized?
> 3. Compare your angle of maximum range with and without air drag. 

### Effect of Air Density

Clearly air drag plays an imporant role in this problem.  However, the plots showed that the shells travel many kilometers up into the air.  The air density will be lower up high in the atmosphere than down at sea level. 

From last lecture we know that air drag is proportional to the density of air. How important will this high altitude effect be on the on the trajectory of the cannon shell?


The details of how air density varies as a function of altitude will be covered in Phys 3400: Thermodynamics.  Here, let's assume that the atmosphere is an *isothermal* (constant temperature) ideal gas.  The pressure $p$ depends on alitude like

$$ p(y) = p(0) \exp(-mgy/k_B T) $$

where $m$ is the average mass of an air molecule, $y$ is the height from some reference point like sea level, $k_B$ is the Boltzmann's constant, and $T$ is the absolute temperature.  For an ideal gas, the pressure so this leads to

$$ \rho = \rho_0 \exp(-y/y_0) $$

where $y_0 = k_B T / m g \approx 1.0 \times 10^4$ m, and $\rho_0$ is the density at sea level ($y=0$).

The air drag force, which is proportional to density, then can be modified to be a function of altitude

$$ F_{drag}^* = \frac{\rho}{\rho_0} F_{drag}(y=0) $$

This value $y_0$ is sometimes called the scale height.  Let's modify our computation steps to include the effect of this scale height.

In [ ]:
def update(i, x, y, vx, vy, Bm, y0=None):
    
    # update positions
    x[i+1] = x[i] + vx*dt
    y[i+1] = y[i] + vy*dt
        
    # calculate velocity magnitude
    v = np.sqrt(vx**2 + vy**2)
        
    # compute air drag
    Fdrag_x = -(Bm)*v*vx
    Fdrag_y = -(Bm)*v*vy
    
    # correct for density change
    if y0 is not None:
        rho_rho0 = np.exp(-y[i]/y0)
        Fdrag_x = rho_rho0 * Fdrag_x
        Fdrag_y = rho_rho0 * Fdrag_y
    
    # update velocities
    vx = vx + Fdrag_x * dt
    vy = vy + (-g + Fdrag_y) * dt
    
    return vx, vy

In [ ]:
def solve(theta=45, v0=700, Bm=4e-5, y0=None):
    """
    Given 
        theta, firing angle (degrees)
        v0, initial velocity (m/s)
        Bm, drag coefficient per unit mass (1/m)
        y0, scale height of atmosphere
    Compute
        x,y trajectory of a cannon shell
    """
    
    # Allocate memory for arrays and set to zero
    N = round(tmax / dt)
    x = np.zeros(N)
    y = np.zeros(N)
    
    # Initial values
    x[0] = 0
    y[0] = 0
    vx = v0*np.cos(np.deg2rad(theta))
    vy = v0*np.sin(np.deg2rad(theta))

    # Calculate the solution
    for i in range(N-1):
        vx, vy = update(i, x, y, vx, vy, Bm, y0)
        
        # stop when y <= 0
        if y[i+1] <= 0:
            break

    # truncate arrays to when shell has hit the ground (y<=0)
    N = i+2
    x = x[:N]
    y = y[:N]

    # interpolate position if shell is below ground level
    if y[i+1] < 0:
        r = -y[i]/y[i+1]
        x[i+1]=(x[i]+r*x[i+1])/(r+1)
        y[i+1]=0 

    return x, y

In [ ]:
fig, axes = plt.subplots()

x, y = solve(theta=45, Bm=4e-5, y0=None)
plot(x, y, style=':', label='Without density correction')
x, y = solve(theta=35, Bm=4e-5, y0=None)
plot(x, y, style=':', label='Without density correction')
x, y = solve(theta=45, Bm=4e-5, y0=1e4)
plot(x, y, label='With density correction')
x, y = solve(theta=35, Bm=4e-5, y0=1e4)
plot(x, y, label='With density correction')
plt.ylim(0,12)
plt.legend(loc='upper left')
plt.show()

#### Exercise
> In our model of the cannon shell trajectory we have assumed that the acceleration due to gravity, $g$, is constant. It will also depend on altitude. Add this to the model and calculate how much it affects the range.

The following formula approximates the Earth's gravity variation with altitude:

$$
g_{h}=g_{0}\left({\frac  {r_{{\mathrm  {e}}}}{r_{{\mathrm  {e}}}+h}}\right)^{2}$$

where

 * $g_h$ is the gravitational acceleration at height h above sea level.
 * $r_e$ is the [Earth's mean radius](https://en.wikipedia.org/wiki/Earth_radius)
 * $g_0$ is the [standard gravitational acceleration](https://en.wikipedia.org/wiki/Standard_gravitational_acceleration)